# Growth approximations plotting code 
Calculates growth approximation values as a function of z and then uses these to plot over the top of the histogram and fitted curve. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy 
import scipy.stats
import scipy.optimize as scopt
from scipy.stats import norm
from scipy.stats import skew
from scipy.stats import skewnorm
from scipy.optimize import curve_fit as scurv


## Create functions to calculate gaussian and growth approximation, and scale density parameters with redshift

In [ ]:
def gaussian(x, mean, amplitude, standard_deviation):
    return amplitude * np.exp( - (x - mean)**2 / (2*standard_deviation ** 2))

In [ ]:
def Omega(z, O_mass, O_lambda):
    return (O_mass*(1+z)**3)/(O_mass*(1+z)**3 + O_lambda)

In [ ]:
def Lahav(O_mass, O_lambda):
    return O_mass**(4/7) + (1 + O_mass/2)*(O_lambda/2)

In [ ]:
def Carroll(O_mass, O_lambda): 
    return (5/2)*O_mass/(O_mass**(4/7) - O_lambda + (1 + O_mass/2)*(1 + O_lambda/70))

## Load data 

In [ ]:
def load_data(rad):
    
    z = []
    deltas = []

    q = open(f'Text_data/z_{rad}.txt', 'r')
    lines = q.readlines()

    for i in lines: 
        z.append(i.split(  )[0])

    q.close()
    z_arr = np.array(z).astype(float)
    sort = np.argsort(z_arr)
    z_arr = z_arr[sort]

    k = open(f'Text_Data/delta_{rad}.txt', 'r')
    lines = k.readlines()

    for i in lines:
        deltas.append(i.split(  ))

    k.close()
    delta_arr = np.array(deltas).astype(float)
    delta_arr = delta_arr[sort]
    
    return delta_arr, z_arr


## Calculate growth rate values and use these to plot over histograms, then save while exporting values

In [ ]:
def approx_plotting(z_arr, delta_arr, rad):
    
    var_arr = np.ones(len(z_arr))
    analytic = np.array([])
    carroll = np.array([])
    lahav = np.array([])

    x = np.linspace(-1,1,10000)
    
    for i in range(len(z_arr)):
        
        
        z = z_arr[i] 
        a = 1/(1+z)
        
        Omega_m = Omega(z, 0.285, 0.715)
        Omega_l = 1 - Omega_m
        
        lahav_approx = Lahav(Omega_m, Omega_l)
        carroll_approx = Carroll(Omega_m, Omega_l)
        
        # Find mean and standard deviation of delta array 
        mean, std = norm.fit(delta_arr[i])

        # Plot histogram of overdensity data and save
        z = np.round(z, decimals = 2)
        n_bin = 500
        plt.clf()
        hist =plt.hist(delta_arr[i], bins = n_bin, range = (-1,1), label = 'Overdensities', color = 'g')
        plt.title(f'Overdensity Histogram for z = {z}')
        plt.xlabel('Overdensity $\delta$')
        plt.ylabel('Frequency')
           
        
        bin_heights, bin_borders = np.histogram(delta_arr[i], bins = n_bin, range = (-1,1))
        bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 
        
        popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
        
        analytic = np.append(analytic, popt[2])

        plt.plot(x, gaussian(x, *popt), label = 'Fitted', color = 'blue')
        
        lahav = np.append(lahav, lahav_approx*a)
        
        plt.plot(x, gaussian(x, popt[0], popt[1], lahav_approx*a), label = 'Lahav', color = 'r')
        
        carroll = np.append(carroll, carroll_approx*a )
        
        plt.plot(x, gaussian(x, popt[0], popt[1], carroll_approx*a), label = 'Carroll', color = 'orange')
        
        plt.xlabel('Overdensity $\delta$')
        plt.ylabel('Frequency')
        plt.xlim(-1,2)
        plt.title(f'Linear Extrapolation of Overdensity Values for z = {z}')
        plt.legend()
        plt.savefig(f'Extrapolated/Extrapolated_{rad}Mpc_redshift_{z}.jpeg', dpi =200)
        plt.show()
        
    return carroll, lahav
    

## Write out growth rate values for use elsewhere

In [ ]:
def data_out(lahav, carroll, rad):     
    outfile = open(f'Text_Data/Extrapolation Data/lahav_{rad}Mpc.txt', 'w')

    for i in range(len(lahav)):
        outfile.write(str(lahav[i]) + "\n")

    outfile.close()

    outfile = open(f'Text_Data/Extrapolation Data/carroll_{rad}Mpc.txt', 'w')

    for i in range(len(carroll)):
        outfile.write(str(carroll[i]) + "\n")

    outfile.close()

In [ ]:
rads = [30]
for rad in rads:
    delta_arr, z_arr = load_data(rad)
    carroll, lahav = approx_plotting(z_arr, delta_arr, rad)
    data_out(lahav, carroll, rad)